# Import Packages

In [ ]:
from torch.utils.data import DataLoader

from utils import *

# Setup

In [ ]:
csv_file = "train.csv"
data_root = "train"
data_list = pd.read_csv(csv_file)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# DataLoader

In [ ]:
train_transform = transforms.Compose([
    JPGLoader(keys=["image"]),
    Resize(keys=["image"], size=(224, 224), interpolation=2),
    PILToTensor(keys=["image"]),
    NumpyToTensor(keys=["label"])
])
val_transform = transforms.Compose([
    JPGLoader(keys=["image"]),
    Resize(keys=["image"], size=(224, 224), interpolation=2),
    PILToTensor(keys=["image"]),
    NumpyToTensor(keys=["label"])
])

train_list = data_list.iloc[:25000, :]
val_list = data_list.iloc[25000:, :]

train_dataset = RANZCR(data_root, train_list, transform=train_transform)
val_dataset = RANZCR(data_root, val_list, transform=val_transform)

data = train_dataset[0]
print(data)

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=True, num_workers=0)

for data in val_loader:
    print(data["image"].shape)
    break

# Model, Criterion, Optimizer

In [ ]:
model = ExampleModel(1, 11).to(device)

x = torch.rand(32, 1, 224, 224).to(device)
y = model(x)
y.size()

In [ ]:
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# Training and Validation

In [ ]:
model.train()
n_train = len(train_loader)
n_val = len(val_loader)

for i, data in enumerate(train_loader):
    optimizer.zero_grad()
    image = data["image"].to(device)
    label = data["label"].to(device)

    prob = model(image)
    loss = criterion(prob, label)
    loss.backward()
    optimizer.step()

    acc = compute_accuracy(prob > 0.5, label)
    print(f"step [{i}, {n_train}], loss = {loss.cpu().detach().numpy()}, acc = {acc.cpu().detach().numpy()}")


model.eval()

with torch.no_grad():
    for i, data in enumerate(val_loader):
        image = data["image"].to(device)
        label = data["label"].to(device)

        prob = model(image)
        loss = criterion(prob, label)
        acc = compute_accuracy(prob > 0.5, label)

        print(f"step [{i}, {n_val}], loss = {loss.cpu().detach().numpy()}, acc = {acc.cpu().detach().numpy()}")